In [5]:
import time
from adafruit_servokit import ServoKit

kit = ServoKit(channels=16)


mapp = {0:0,1:130,2:125}
# Reset all standard servos
# for ch in [7,13,14,15]:
#     try:
#         kit.servo[ch].angle = mapp[ch]  # release (no signal)
#     except Exception:
#         pass

# Reset all continuous servos
for ch in range(16):
    try:
        kit.continuous_servo[ch].throttle = 0  # stop
        time.sleep(0)

        # or use None to release completely
    except Exception:
        pass

print("✅ All channels reset to neutral/off")

✅ All channels reset to neutral/off


In [ ]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50  # standard servo frequency
kit = ServoKit(channels=16)  # For standard small servos

# -----------------------------
# Servo configuration
# -----------------------------
# Define which channels have large/high-torque vs small/RC servos
large_servos = [5,6]          # high-torque / metal gear
small_servos = [1, 3, 4] # standard hobby/RC servos

# Create servo objects for large servos with wide pulse range
large_servo_objects = {}
for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s
    s.angle = 0  # initialize to center
    time.sleep(0.05)

time.sleep(1)  # give all large servos time to reach position

# -----------------------------
# Helper functions
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    """
    Gradually move a large servo to the target angle
    """
    target = max(0, min(180, int(round(target))))
    if target == current:
        return current

    step = max(1, int(abs(step)))

    if target > current:
        angle = current
        while angle < target:
            angle = min(angle + step, target)
            servo_obj.angle = angle
            time.sleep(delay)
    else:
        angle = current
        while angle > target:
            angle = max(angle - step, target)
            servo_obj.angle = angle
            time.sleep(delay)

    return target  # updated current angle

def sweep_large_servo(ch):
    """
    Sweep a large servo on channel `ch` using gradual steps
    """
    s = large_servo_objects[ch]
    current_angle = 90
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 180, step=3, delay=0.04)
    current_angle = move_to(s, current_angle, 90, step=2, delay=0.04)
    current_angle = move_to(s, current_angle, 0, step=3, delay=0.04)

def sweep_small_servo(ch):
    """
    Sweep a small hobby/RC servo using direct jumps
    """
    print(f"Sweeping small servo on channel {ch}")
    
    for angle in [90, 180, 90, 0, 90]:
        kit.servo[ch].angle = angle
        time.sleep(1)  # give servo time to reach position

# -----------------------------
# Main program
# -----------------------------
try:
    print("=== Sweeping LARGE servos ===")
    for ch in large_servos:
        print(f"Channel {ch}")
        sweep_large_servo(ch)

    # print("\n=== Sweeping SMALL servos ===")
    # for ch in small_servos:
    #     sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    # pca.deinit()


=== Sweeping LARGE servos ===
Channel 5
Channel 6

Cleaning up PCA9685


In [46]:
# 1 - j4 - mouth/hand/gripper
# 3 - j3 - wrist
# 4 - j2 - elbow
# 5 - j1 - shoulder
# 6 - j0 - waist
from adafruit_servokit import ServoKit

kit = ServoKit(channels=16)

# Dictionary to store last commanded positions
positions = {}

for ch in [1,3,4,5,6]:
    try:
        positions[ch] = kit.servo[ch].angle  # This is last set angle
    except Exception:
        positions[ch] = None

print(positions)

{1: 209.86579910532737, 3: -30.231801545343636, 4: -30.231801545343636, 5: 132.56608377389182, 6: 89.81699877999186}


In [ ]:
import time
import board
import busio
from adafruit_pca9685 import PCA9685
from adafruit_motor import servo
from adafruit_servokit import ServoKit

# Startup angles from your INIT printout
startup_angles = {
    6: 90,
    5: 180,
    4: 0,
    3: 0,
    1: 180.0
}

# -----------------------------
# PCA9685 setup
# -----------------------------
i2c = busio.I2C(board.SCL, board.SDA)
pca = PCA9685(i2c)
pca.frequency = 50
kit = ServoKit(channels=16,i2c=i2c)

# -----------------------------
# Servo groups
# -----------------------------
large_servos = [5,6]
small_servos = [1, 3, 4]

# -----------------------------
# Create large servo objects
# -----------------------------
large_servo_objects = {}

for ch in large_servos:
    s = servo.Servo(pca.channels[ch], min_pulse=500, max_pulse=2500)
    large_servo_objects[ch] = s

    # set to real startup angle
    sa = max(0, min(180, int(startup_angles[ch])))
    s.angle = sa
    time.sleep(0.05)

time.sleep(0.3)


# -----------------------------
# Smooth movement helper
# -----------------------------
def move_to(servo_obj, current, target, step=2, delay=0.02):
    target = max(0, min(180, int(target)))
    current = max(0, min(180, int(current)))
    direction = 1 if target > current else -1

    for a in range(current, target + direction, direction * step):
        servo_obj.angle = a
        time.sleep(delay)

    return target


# -----------------------------
# Determine RELATIVE sweep path
# -----------------------------
def generate_sweep_targets(start):
    if start < 40:
        # near zero → go high first
        return [start, 180, 0, start]

    if start > 140:
        # near 180 → go low first
        return [start, 0, 180, start]

    # middle positions → symmetrical sweep
    return [start, 0, 180, start]


# -----------------------------
# Sweep functions
# -----------------------------
def sweep_large_servo(ch):
    start = startup_angles[ch]
    start = max(0, min(180, int(start)))
    s = large_servo_objects[ch]

    targets = generate_sweep_targets(start)

    print(f"[LARGE] ch{ch} sweep path: {targets}")

    current = targets[0]
    for target in targets[1:]:
        current = move_to(s, current, target, step=2, delay=0.05)


def sweep_small_servo(ch):
    start = startup_angles[ch]
    start = max(0, min(180, int(start)))

    targets = generate_sweep_targets(start)

    print(f"[SMALL] ch{ch} sweep path: {targets}")

    current = start
    for target in targets[1:]:
        kit.servo[ch].angle = target
        time.sleep(0.5)
        current = target


# -----------------------------
# MAIN PROGRAM
# -----------------------------
try:
    print("\n=== Sweeping LARGE servos with RELATIVE logic ===")
    for ch in large_servos:
        sweep_large_servo(ch)

    # print("\n=== Sweeping SMALL servos with RELATIVE logic ===")
    # for ch in small_servos:
    #     sweep_small_servo(ch)

finally:
    print("\nCleaning up PCA9685")
    # pca.deinit()



=== Sweeping LARGE servos with RELATIVE logic ===
[LARGE] ch5 sweep path: [180, 0, 180, 180]


In [44]:
import time
import board
import busio
from adafruit_servokit import ServoKit

# --- Configuration ---
STARTUP_ANGLES = {
    6: 90.0,
    5: 180.0,
    4: 0.0,
    3: 0.0,
    1: 180.0
}

LARGE_SERVOS = [5, 6]
SMALL_SERVOS = [1, 3, 4]

MIN_PULSE = 500
MAX_PULSE = 2500
SMOOTH_STEP = 1
SMOOTH_DELAY = 0.05

# --- PCA9685 Setup via ServoKit ---
i2c = board.I2C()  # auto-detects SCL/SDA
kit = ServoKit(channels=16, i2c=i2c, frequency=50)

# --- Initialize All Servos ---
all_channels = LARGE_SERVOS + SMALL_SERVOS
for ch in all_channels:
    kit.servo[ch].set_pulse_width_range(MIN_PULSE, MAX_PULSE)
    start_angle = max(0.0, min(180.0, STARTUP_ANGLES.get(ch, 90.0)))
    kit.servo[ch].angle = start_angle
    time.sleep(0.05)

print("Initialization complete. Servos at startup angles.")
time.sleep(0.3)

# --- Helper Functions ---
def move_to_smoothly(servo_obj, current, target, step=SMOOTH_STEP, delay=SMOOTH_DELAY):
    """Smoothly moves a servo from current to target angle."""
    target = max(0, min(180, target))
    current = max(0, min(180, current))
    direction = 1 if target > current else -1

    if abs(target - current) < step:
        servo_obj.angle = target
        return target

    for angle in range(round(current), round(target) + direction * step, direction * step):
        servo_obj.angle = float(angle)
        time.sleep(delay)

    servo_obj.angle = target
    return target

def generate_sweep_targets(start):
    """Generate sweep path relative to start angle."""
    start = round(start)
    if start < 40:
        return [start, 180, 0, start]
    if start > 140:
        return [start, 0, 180, start]
    return [start, 0, 180, start]

# --- Sweep Functions ---
def sweep_large_servo(ch):
    start = STARTUP_ANGLES[ch]
    targets = generate_sweep_targets(start)
    print(f"[LARGE] ch{ch} sweep path: {targets}")
    current = targets[0]
    for target in targets[1:]:
        current = move_to_smoothly(kit.servo[ch], current, target)

def sweep_small_servo(ch):
    start = STARTUP_ANGLES[ch]
    targets = generate_sweep_targets(start)
    print(f"[SMALL] ch{ch} sweep path: {targets}")
    current = start
    for target in targets[1:]:
        kit.servo[ch].angle = target
        time.sleep(0.5)
        current = target

# --- Main Execution ---
try:
    print("\n=== Sweeping LARGE Servos ===")
    for ch in LARGE_SERVOS:
        sweep_large_servo(ch)

    print("\n=== Sweeping SMALL Servos ===")
    # for ch in SMALL_SERVOS:
    #     sweep_small_servo(ch)

finally:
    print("\nSweep complete. All servos at final positions.")


Initialization complete. Servos at startup angles.

=== Sweeping LARGE Servos ===
[LARGE] ch5 sweep path: [180, 0, 180, 180]
[LARGE] ch6 sweep path: [90, 0, 180, 90]

=== Sweeping SMALL Servos ===

Sweep complete. All servos at final positions.


In [30]:
!sudo apt-get install python-smbus
!sudo apt-get install i2c-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package python-smbus is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'python-smbus' has no installation candidate
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
i2c-tools is already the newest version (4.3-2+b3).
The following packages were automatically installed and are no longer required:
  avahi-utils gir1.2-handy-1 gir1.2-packagekitglib-1.0 gir1.2-polkit-1.0
  libbasicusageenvironment1 libc++1-16 libc++abi1-16 libcamera0.3 libcamera0.4
  libgroupsock8 liblivemedia77 libunwind-16 libwlroots12
  linux-headers-6.6.51+rpt-common-rpi linux-headers-6.6.51+rpt-rpi-2712
  linux-headers-6.6.51+rpt-rpi-v8 linux-image-6.6.51+rpt-rpi-2712
  linux-image-6.6.51+rpt-rpi-v8 linux-kbuild-6.6.51+rpt lxplug-network
  python3-v4l2
Us

In [2]:
!sudo i2cdetect -y 1


     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:                         -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: -- -- -- -- -- -- -- --                         


In [ ]:
from adafruit_servokit import ServoKit
kit = ServoKit(channels=16)

kit.servo[5].__dict__
kit.servo[5].angle=180
kit.servo[5].angle



180.0